In [141]:
import argparse
import pywt
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy.fftpack import dct, idct
import numpy as np

def handle_input(filename, matrix_mode = False):
    '''
        Converts filename into rgb matrix (or rgba depending on input image)
        input   : string => filename
        ouptut  : [[(r,g,b)]] or [[(r,g,b,a)]] => 2d matrix where each element is a tuple of r, g, b (or RGBA)
    '''
    if matrix_mode:
        return read_as_fp(filename[:filename.rfind('.')+1]+'txt')
    return io.imread(filename)/255
    

def handle_output(rgb_matrix, output_filename, matrix_mode = False):
    '''
        Converts rgb (or rgba depending on input image) matrix into image file
        input    : [[(r,g,b)]] or [[(r,g,b,a)]] => 2d matrix where each element is a tuple of r, g, b (or RGBA)
                   string => filename
    '''
    if matrix_mode:
        save_as_fp(rgb_matrix, output_filename[:output_filename.rfind('.')+1]+'txt')
    io.imsave(output_filename,rgb_matrix)

def handle_var_size(placeholder, watermark):
    '''
        Resizes the images based on the size of placeholder and watermark, making them have the same shape
        input    : placeholder and watermark image matrix
    '''
    M = max(placeholder.shape[0], watermark.shape[0])
    N = max(placeholder.shape[1], watermark.shape[1])
    
    placeholder_resized = resize(placeholder, (M, N), anti_aliasing=True)
    watermark_resized = resize(watermark, (M, N), anti_aliasing=True)
    return placeholder_resized, watermark_resized  

def handle_var_resize(image, size):
    '''
        Resizes the images based on the size of placeholder and watermark, making them have the same shape
        input    : placeholder and watermark image matrix
    '''
    image_resized = resize(image, (size, size), 1)
    return image_resized

def rgb_to_grey(rgb_img):
    return rgb2gray(rgb_img)

def apply_by_block(fn, image, block_size=8):
    img_dct = np.empty((len(image[0]), len(image[0])))
    for i in range (0, len(image[0]), block_size):
        for j in range (0, len(image[0]), block_size):
            current_block = image[i:i+block_size, j:j+block_size]
            current_block_dct = fn(fn(current_block.T, norm="ortho").T, norm="ortho")
            img_dct[i:i+block_size, j:j+block_size] = current_block_dct
    return img_dct

def embed_by_block(img_dct, to_hide_array, block_size=8):
    idx = 0
    for i in range (0, len(img_dct), block_size):
        for j in range (0, len(img_dct), block_size):
            if idx < len(to_hide_array):
                current_dct = img_dct[i:i+block_size, j:j+block_size]
                current_dct[5][5] = to_hide_array[idx]
                img_dct[i:i+block_size, j:j+block_size] = current_dct
                idx += 1 
    return img_dct

def extract_by_block(img_dct, block_size=8):
    combined_watermark = []
    for x in range (0, len(img_dct), block_size):
        for y in range (0, len(img_dct), block_size):
            inner_dct = img_dct[x:x+block_size, y:y+block_size]
            combined_watermark.append(inner_dct[5][5])
    return combined_watermark

def encrypt_dctdwt(placeholder_image, to_hide_image, encryption_method, output_filename='watermarked_image.png'):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : string => placeholder filename
                   string => filename of image to hide
                   string => encryption method chosen
                   string => filename of output image
    '''
    
    #Getting Image matrices
    placeholder_rgb = handle_input(placeholder_image)
    to_hide_rgb = handle_input(to_hide_image)
    
    #Resizing to correct sizes
    placeholder_rgb = handle_var_resize(placeholder_rgb, 2048)
    to_hide_rgb = handle_var_resize(to_hide_rgb, 128)
    
    #Converting rgb to greyscale
    placeholder_rgb = rgb_to_grey(placeholder_rgb)
    to_hide_rgb = rgb_to_grey(to_hide_rgb)
    
    #Watermarking starts here
    
    #Applying 2D multilevel Discrete Wavelet Transform
    wt_l1 = list(pywt.wavedec2(data = placeholder_rgb, wavelet = 'haar', level = 1))
    
    #Applying DCT 
    dct_l1 = apply_by_block(dct, wt_l1[0], 8)
    
    #Embed watermark into image
    dct_l1 = embed_by_block(dct_l1, to_hide_rgb.ravel(), 8)
    
    #Applying IDCT
    wt_l1[0] = apply_by_block(idct, dct_l1, 8)
    
    #2D multilevel reconstruction 
    watermarked_img = pywt.waverec2(wt_l1, 'haar')
    
    #Watermarking ends here
    
    #Saving the output watermarked image
    #io.imshow(watermarked_img)
    handle_output(watermarked_img, output_filename)
    
def decrypt_dctdwt(encrypted_image, output_filename = 'watermark.png'):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : string => filename of encrypted image
                   string => encryption method chosen
                   string => filename of output image
    '''
    
    #Getting image matrix 
    encrypted_rgb = handle_input(encrypted_image, True)
    
    #Resizing to correct sizes
    encrypted_rgb = handle_var_resize(encrypted_rgb, 2048)
    
    #Applying 2D multilevel Discrete Wavelet Transform
    wt_l1 = list(pywt.wavedec2(data = encrypted_rgb, wavelet = 'haar', level = 1))
    
    #Applying DCT by block size of 8
    dct_l1 = apply_by_block(dct, wt_l1[0], 8)
    
    #Extracting watermark
    combined_watermark = extract_by_block(dct_l1, 8)
    decrypted_img = np.array(combined_watermark).reshape(128, 128)
    
    #Saving the output watermark image
    handle_output(decrypted_img, output_filename)

def enc_dct(encryption_parameter, placeholder_image, to_hide_image, encryption_method, output_filename):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : float  => encryption parameter
                   string => placeholder filename
                   string => filename of image to hide
                   string => encryption method chosen
                   string => filename of output image
    '''
    
    placeholder_rgb = handle_input(placeholder_image)
    to_hide_rgb = handle_input(to_hide_image)
    placeholder_rgb, to_hide_rgb = handle_var_size(placeholder_rgb, to_hide_rgb)

    dct_image = dct(placeholder_rgb, norm='ortho') + (encryption_parameter * to_hide_rgb)
    encrypted_rgb = idct(dct_image, norm='ortho')
    
    handle_output(encrypted_rgb, output_filename, True)
    
def decrypt_dct(encryption_parameter, placeholder_image, encrypted_image, encryption_method, output_filename):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : string => filename of encrypted image
                   string => encryption method chosen
                   string => filename of output image
    '''
    
    placeholder_rgb = handle_input(placeholder_image)
    encrypted_rgb = handle_input(encrypted_image, True)
   
    decrypted_rgb = (dct(encrypted_rgb, norm='ortho') - dct(placeholder_rgb, norm='ortho'))/encryption_parameter
    
    handle_output(decrypted_rgb, output_filename)
    
def encrypt(placeholder_image, to_hide_image, encryption_method, output_filename):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : string => placeholder filename
                   string => filename of image to hide
                   string => encryption method chosen
                   string => filename of output image
    '''

    if(encryption_method == 'dct'):
        encryption_parameter = 0.01
        enc_dct(encryption_parameter, placeholder_image, to_hide_image, encryption_method, output_filename)
        
    elif(encryption_method == 'dctdwt'):
        encrypt_dctdwt(placeholder_image, to_hide_image, encryption_method, output_filename)
        
    else:
        print("Encryption Method not specified or unavailable")
    
def decrypt(placeholder_image, encrypted_image, encryption_method, output_filename):
    '''
        Encrypts to_hide_image into placeholder method using encryption_method and outputs encrypted file as output_filename
        input    : string => filename of encrypted image
                   string => encryption method chosen
                   string => filename of output image
    '''
    
    if(encryption_method == 'dct'):
        encryption_parameter = 0.01
        decrypt_dct(encryption_parameter, placeholder_image, encrypted_image, encryption_method, output_filename)
    
    elif(encryption_method == 'dctdwt'):
        decrypt_dctdwt(encrypted_image, output_filename)
        
    else:
        print("Encryption Method not specified or unavailable")

In [142]:
def save_as_fp(matrix, filename):
    '''
        Writes the matrix with fp to a .txt file to prevent loss
        input    : numpy.ndarray => 2d matrix where each element is a tuple of r, g, b (or RGBA)
                   string => filename of output txt file
    '''
    file = open(filename, "w", encoding ="utf-8")
    
    #Store the shape of the matrix
    file.write(' '.join([str(elem) for elem in matrix.shape]) + '\n')
    
    #Store the pixel values with floating point
    for row in matrix:
        for tup in row:
            file.write(' '.join([str(elem) for elem in tup]) + '\n')
            
    file.close()
    
def read_as_fp(filename):
    '''
        Writes the matrix with fp to a .txt file to prevent loss
        input    : numpy.ndarray => 2d matrix where each element is a tuple of r, g, b (or RGBA)
                   string => filename of output txt file
    '''
    file = open(filename, "r", encoding ="utf-8")
    textString = file.read().splitlines()
    shape = textString[0].split()
    num_of_rows = int(shape[0])
    num_of_tuples = int(shape[1])
    num_of_elems = int(shape[2])
    tempArr = list()
    for s in textString[1:]:
        for num in s.split():
            tempArr.append(float(num))
    array = np.array(tempArr).reshape(num_of_rows, num_of_tuples, num_of_elems)
    print(array.dtype.type)
    return array
    #matrix = numpy.ndarray((num_of_rows, num_of_tuples, num_of_elems), dtype=float, buffer=None, offset=0, strides=None, order=None)
    
    '''
    start = 1
    for 
    for row in file.read().splitlines()[start:num_of_tuples]:
        tup = (float(row[0]), float(row[1]), float(row[2]))
        
    #Store the shape of the matrix
    file.write(' '.join([str(elem) for elem in matrix.shape]) + '\n')
    
    #Store the pixel values with floating point
    for row in matrix:
        for tup in row:
            file.write(' '.join([str(elem) for elem in tup]) + '\n')
            
    file.close()
    '''

In [143]:
%%time
encrypt('bunny.png', 'cube.png', 'dct', 'test1.png')

Lossy conversion from float64 to uint8. Range [0.0027157148300435627, 1.0013409734542598]. Convert image to uint8 prior to saving to suppress this warning.


Wall time: 11.4 s


In [131]:
#%%time
#read_as_fp('abc.txt') #5269504

<class 'numpy.float64'>
Wall time: 4.74 s


array([[[0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        ...,
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192]],

       [[0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        ...,
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192]],

       [[0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        ...,
        [0.12229808, 0.11143259, 0.12386152, 0.99770192],
        [0.12229808, 0.111432

In [144]:
%%time
decrypt('bunny.png','test1.png','dct','hidden_chocolate.png')

<class 'numpy.float64'>


Lossy conversion from float64 to uint8. Range [-9.14823772291129e-12, 1.000000000004947]. Convert image to uint8 prior to saving to suppress this warning.


Wall time: 5.72 s
